In [33]:
# Author: Riley Owens (GitHub: mrileyowens)

# This file measures values and errors of 
# various Lyα profile parameters.

In [34]:
import os

from math import floor, log10

from datetime import datetime, timezone

import numpy as np

from scipy.optimize import curve_fit
from scipy.special import erf
from scipy.stats import kendalltau

import sigfig

from astropy.cosmology import FlatLambdaCDM

import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText

In [ ]:
def fit():

    '''
    Directly fit models of the Lya profiles in a Monte Carlo simulation
    '''

    def extract_data(file):

        '''
        Extract spectrum from the .txt file

        Parameters:
            file : str
                Name of the file

        Returns:
            w : numpy.ndarray
                Observed wavelength bins
            f : numpy.ndarray
                Observed flux densities
            n : numpy.ndarray
                Observed Gaussian standard deviation of observed flux densities
        '''

        # Retrieve the data columns
        w, f, n = np.loadtxt(file, delimiter='\t', comments=('#', 'w'), usecols=(0,1,2), unpack=True)
    
        # If the file is not the stacked leaker spectrum. This step is necessary because 
        if 'leaker' not in file:

            # Remove bins of extreme outliers
            w = w[f < 1e-20]
            n = n[f < 1e-20]
            f = f[f < 1e-20]

            # Convert from units of erg/s/cm^2/Hz to erg/s/cm^2/Å
            f = f * 2.998e18 / np.square(w)
            n = n * 2.998e18 / np.square(w)

        return w, f, n    

    def compute_fwhm_and_peak(parameters, Rs):

        '''
        Compute the FWHMs and locations of the maxima of a set of fits of a Lya peak

        Parameters:

            parameters : numpy.ndarray
                The best-fit model parameters from the Monte Carlo simulation
            Rs : numpy.ndarray
                Spectral resolutions of the sets of parameters

        Returns:
            
            fwhms : numpy.ndarray
                The FWHMs of the Lya peaks corresponding to the input parameters
            locs : numpy.ndarray
                The location of the maxima of the Lya peaks corresponding to the input parameters
        '''

        def lin_interp(x, y, i):
            return x[i] + (x[i+1] - x[i]) * y[i] / (y[i+1] - y[i])

        def skew_gaussian(x, amp, cen, width, skew):

            fit = amp * np.exp(-((x - cen) / width)**2 / 2) * (1 + erf(skew * ((x - cen) / width) / np.sqrt(2)))

            return fit

        # Make a an array of velocity bins; we are not using the data's bins 
        # because they are a large fraction of the FWHMs of the peaks
        v = np.arange(-500,500,0.1)

        # Make an empty list to append the best-fit curves to
        fits = []

        # For each set of best-fit parameters of the Lya peak
        for i, fit_params in enumerate(parameters):

            # Create the model curve
            fit = skew_gaussian(v, *fit_params)

            # Append the curve to the aggregate list
            fits.append([*fit])

        # Calculate the velocity locations of the maxima of the peaks
        fits_max_indices = np.argmax(fits, axis=1)
        locs = v[fits_max_indices]

        # Calculate the fits halved by the maximum of each peak
        fits_halved = fits - np.amax(fits, axis=1, keepdims=True) / 2

        # Find where the halved fits cross the x-axis
        crossings = np.diff(np.sign(fits_halved), axis=1) != 0

        # Get the indices of the x-axis crossings
        indices = np.array(np.where(crossings)).T

        # Make an empty list to append the FWHMs to
        fwhms = []

        # For each set of parameters
        for i in range(len(parameters)):

            # Calculate the FWHM of the peak by linearly interpolating the location of the zero-crossing points of the halved fits
            fwhm = abs(lin_interp(v, fits_halved[i], indices[2 * i][1]) - lin_interp(v, fits_halved[i], indices[2 * i + 1][1]))

            # Append the FWHM to the aggregate list
            fwhms.append(fwhm)

        # Correct the FWHMs for the instrumental FWHM
        fwhms = np.sqrt((np.array(fwhms))**2 - (Rs)**2)

        # Replace any NaNs (where the instrumental FWHM was greater than the measured FWHM) with just the instrumental FWHMs
        fwhms = np.where(np.isnan(fwhms), Rs, fwhms)

        return fwhms, locs

    def two_peaks(x, amp_r, cen_r, width_r, skew_r, amp_c, cen_c, width_c, cntm):

        return amp_r * np.exp(-((x - cen_r) / width_r)**2 / 2) * (1 + erf(skew_r * ((x - cen_r) / width_r) / np.sqrt(2))) \
            + amp_c * np.exp(-((x - cen_c) / width_c)**2 / 2) \
            + cntm

    def three_peaks(x, amp_b, cen_b, width_b, skew_b, amp_r, cen_r, width_r, skew_r, amp_c, cen_c, width_c, cntm):

        return amp_b * np.exp(-((x - cen_b) / width_b)**2 / 2) * (1 + erf(skew_b * ((x - cen_b) / width_b) / np.sqrt(2)))  \
            + amp_r * np.exp(-((x - cen_r) / width_r)**2 / 2) * (1 + erf(skew_r * ((x - cen_r) / width_r) / np.sqrt(2))) \
            + amp_c * np.exp(-((x - cen_c) / width_c)**2 / 2) \
            + cntm

    # Dictionary containing information about the MagE slit spectra, including, from left to right:
    # the name of the file containing the data, redshift (taken from Mainali et al. (2022) (ApJ, 940, 160)),
    # magnification of the slit, a tuple specifying a range of Lya peculiar velocities used to set 
    # initial parameter estimates of the central Lya peak, spectral resolution, and uncertainty in the spectral resolution
    slits = {
        'NL' : ['sunburst_arc_nonleaker_stack_mage.txt', 0, 1, [60,140], 5400, 200],
        'L' : ['sunburst_arc_leaker_stack_mage.txt', 0, 1, [20,130], 5300, 200],
        'M5' : ['sunburst_M-5-comb1_MWdr.txt', 2.37086, 51, [0,100], 5500, 400],
        'M4' : ['sunburst_M-4-comb1_MWdr.txt', 2.37073, 14.6, [0,85], 5400, 300],
        'M6' : ['sunburst_M-6-comb1_MWdr.txt', 2.37021, 147, [10,130], 5300, 300],
        'M3' : ['sunburst_M-3-comb1_MWdr.txt', 2.37025, 36, [35,120], 5500, 400],
        'M0' : ['sunburst_M-0-comb1_MWdr.txt', 2.37014, 10, [10,130], 4700, 200],
        'M2' : ['sunburst_M-2-comb1_MWdr.txt', 2.37017, 32, [45,125], 5300, 300],
        'M7' : ['sunburst_M-7-comb1_MWdr.txt', 2.37044, 35, [15,100], 5200, 200],
        'M8' : ['sunburst_M-8-comb1_MWdr.txt', 2.37024, 29, [25,125], 5200, 300],
        'M9' : ['sunburst_M-9-comb1_MWdr.txt', 2.37030, 31, [15,125], 5500, 400]
    }

    # Establish directories
    home = os.getcwd()
    data = f'{home}/data'
    results = f'{home}/results'

    # Instantiate a cosmology with 30% matter-based energy density and an expansion rate of 70 km/s/Mpc
    cosmology = FlatLambdaCDM(70,0.3)

    # For each slit ID
    for i, slit_id in enumerate(slits):

        # Unpack the dictionary values for this slit
        file_name, z, mag, c_peak_range, R, R_error = slits[slit_id]

        # Convert the spectral resolution to units of km/s, which is 
        # necessary when correcting the FWHMs (which has units of km/s)
        # for the instrumental resolution
        R_error = 299792.458 / R**2 * R_error
        R = 299792.458 / R

        # Set a scaling factor which will be applied to the data. This is necesary
        # because for the observed data (so not the stacked spectra), the fits are 
        # very poor (the fitting does not explore the parameter space) without rescaling 
        # them, perhaps related to float precision because given their flux density units, 
        # the flux density values are very small (~10^16 - 10^17 erg/s/cm^2/Å)
        factor = 1e16 if 'M' in slit_id else 1

        # Extract the data from the .txt file
        w, f, n = extract_data(f'{data}/mage/{file_name}')

        # Cut the data to just the relevant portion of the spectrum
        f = f[(w >= 1195 * (1 + z)) & (w <= 1235 * (1 + z))]
        n = n[(w >= 1195 * (1 + z)) & (w <= 1235 * (1 + z))]
        w = w[(w >= 1195 * (1 + z)) & (w <= 1235 * (1 + z))]
        
        # Place the data in the rest frame, and apply the scaling factor
        w = w / (1 + z)
        f = f * (1 + z) * factor
        n = n * (1 + z) * factor

        # Calculate the Lya peculiar velocities
        v = 299792.458 * (w / 1215.67 - 1)

        # Determine the luminosity distance to the redshift in units of centimeters (to match the flux density unit)
        l_dist = cosmology.luminosity_distance(z).value * 3.086e24

        # Create a boolean mask designating the wavelength range used to compute the local continuum
        cntm_mask = (w >= 1221) & (w <= 1225)

        # Create a boolean mask designating the integration range when computing the Lya equivalent width and luminosity
        lya_profile_mask = (w >= 1212) & (w <= 1221)
        
        # Create boolean masks for the integration ranges about the rest velocity of the Lya profile
        # for computing the central escape fraction of the Lya profile
        v100_mask = (v >= -100) & (v <= 100)
        v1000_mask = (v >= -1000) & (v <= 1000)

        # Set a string for the flux density unit of the data
        flux_density_unit = 'erg/s/cm^2/Å' if 'M' in slit_id else 'normalized wavelength-space flux density'

        # If there is no blue peak and the slit ID is not M5
        if i in [0,3,4]:
        
            # Set the model function and the model parameter labels
            model = two_peaks
            model_parameter_labels = f'redshifted Lyα peak amplitude ({flux_density_unit}), centroid (km/s), width (km/s), and skew, \n' \
                + f'central Lyα peak amplitude ({flux_density_unit}), centroid (km/s), and width (km/s), ' \
                + f'and local continuum ({flux_density_unit})'
        
            # Set the initial parameter estimates and bounds
            p0 = (np.amax(f[(v >= 150) & (v <= 1000)]),200,20,1,np.amax(f[(v >= c_peak_range[0]) & (v <= c_peak_range[1])]),80,40,np.median(f[cntm_mask]))
            bounds = ([0,0,0,0,0,0,0,0],[np.amax(f),1000,np.inf,np.inf,np.amax(f),200,85,np.amax(f)])
    
        # If the slit ID is M5; this elif statement is necessary for improved initial parameter estimates of the central Lya peak in slit M5. The previous
        # if statement was not satisfactory
        elif i in [2]:

            # Set the model function and the model parameter labels
            model = two_peaks
            model_parameter_labels = f'redshifted Lyα peak amplitude ({flux_density_unit}), centroid (km/s), width (km/s), and skew, \n' \
                + f'central Lyα peak amplitude ({flux_density_unit}), centroid (km/s), and width (km/s), ' \
                + f'and local continuum ({flux_density_unit})'

            # Set the initial parameter estimates and bounds
            p0 = (np.amax(f[(v >= 150) & (v <= 1000)]),200,20,1,np.amax(f[(v >= c_peak_range[0]) & (v <= c_peak_range[1])]) - np.median(f[cntm_mask]),50,20,np.median(f[cntm_mask]))
            bounds = ([0,0,0,0,0,0,0,0],[np.amax(f),500,np.inf,np.inf,np.amax(f[(v >= c_peak_range[0]) & (v <= c_peak_range[1])]) - np.median(f[cntm_mask]),100,40,np.amax(f)])

        # Otherwise
        else:

            # Set the model function and the model parameter labels
            model = three_peaks
            model_parameter_labels = f'blueshifted Lyα peak amplitude ({flux_density_unit}), centroid (km/s), width (km/s), and skew, \n' \
                + f'redshifted Lyα peak amplitude ({flux_density_unit}), centroid (km/s), width (km/s), and skew, \n' \
                + f'central Lyα peak amplitude ({flux_density_unit}), centroid (km/s), and width (km/s), ' \
                + f'and local continuum ({flux_density_unit})'

            # Set the initial parameter estimates and bounds
            p0 = (np.amax(f[(v >= -1000) & (v <= 0)]),-150,20,-1,np.amax(f[(v >= 150) & (v <= 1000)]),200,20,1,np.amax(f[(v >= c_peak_range[0]) & (v <= c_peak_range[1])]),80,40,np.median(f[cntm_mask]))
            bounds = ([0,-1000,0,-np.inf,0,0,0,0,0,0,0,0],[np.amax(f[(v >= -1000) & (v <= 0)]),0,np.inf,0,np.amax(f),1000,np.inf,np.inf,np.amax(f),200,85,np.amax(f)])

        # Establish empty lists to append the results of the burn-in Monte Carlo simulation to
        mc_spectra_burn_in = []
        parameters_burn_in = []

        # For each iteration in the burn-in Monte Carlo simulation
        for j in range(1000):

            # Draw a randomly sampled spectrum from the original observation, assuming 
            # that the observed flux densities and associated uncertainties correspond 
            # to the mean and standard deviation of Gaussian distributions, respectively
            f_mc = np.random.normal(f, n)

            # Directly fit the Lya profile with the assigned model function and initial parameter estimates and bounds
            p, _ = curve_fit(model, v[v >= -500], f_mc[v >= -500], p0=p0, bounds=bounds, maxfev=1e6) 

            # Append the randomly sampled spectrum and best-fit model parameters to the aggregate lists
            mc_spectra_burn_in.append([*f_mc])
            parameters_burn_in.append([*p])

        # Make an array out of the burn-in Monte Carlo simulation spectra for easier handling
        mc_spectra_burn_in = np.array(mc_spectra_burn_in, dtype=np.float64)

        # Make arrays out of the initial and best-fit model parameters for easier handling
        p0 = np.array(p0, dtype=np.float64)
        parameters_burn_in = np.array(parameters_burn_in, dtype=np.float64)

        # Unscale the affected intial model parameter estimates by the artificial scale factor
        p0[-1], p0[-4], p0[-8] = p0[-1] / factor, p0[-4] / factor, p0[-8] / factor

        # Also unscale the initial estimate of the amplitude of the blueshifted Lya peak if the model function had 3 peaks
        p0[0] = p0[0] / factor if model==three_peaks else p0[0]

        # Set the header for the output file containing the randomly sampled Lya spectra of the burn-in Monte Carlo simulation
        header = f'Randomly sampled Lyα spectra of the burn-in Monte Carlo simulation of {f"slit {slit_id}" if "M" in slit_id else slit_id}.\n' \
            + f'Created by lya.ipynb on {datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")}.\n' \
            + '\n' \
            + f'Number of iterations of the burn-in Monte Carlo simulation: {len(mc_spectra_burn_in)}\n' \
            + '\n' \
            + f'Columns, from left to right: peculiar velocity relative to Lyα (km/s), flux density uncertainty ({flux_density_unit}), and randomly sampled flux densities of each iteration of the Monte Carlo simulation ({flux_density_unit})\n'

        # Save the output file containing the randomly sampled Lya spectra of the burn-in Monte Carlo simulation
        np.savetxt(f'{results}/lya_fits/{slit_id}/{slit_id}_mc_sim_burn_in_lya_spectra.txt', np.array([v, n / factor, *(mc_spectra_burn_in / factor)]).T, header=header, delimiter=' ', encoding='utf-8')

        # Set the header for the output file containing the best-fit model parameters of the burn-in Monte Carlo simulation
        header = f'Best-fit model parameters of the Lyα fits of the burn-in Monte Carlo simulation of {f"slit {slit_id}" if "M" in slit_id else slit_id}.\n' \
            + f'Created by lya.ipynb on {datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")}.\n' \
            + '\n' \
            + f'Initial parameters used by the burn-in Monte Carlo simulation: {p0}\n' \
            + f'Number of iterations of the burn-in Monte Carlo simulation: {len(mc_spectra_burn_in)}\n' \
            + '\n' \
            + f'Columns, from left to right: {model_parameter_labels}\n'
        
        # Make a copy of the burn-in best-fit model parameters to manipulate while preserving the original version
        p0_burn_in_save = parameters_burn_in.copy()
        
        # Unscale the affected intial model parameter estimates by the artificial scale factor
        p0_burn_in_save[:,-1], p0_burn_in_save[:,-4], p0_burn_in_save[:,-8] = p0_burn_in_save[:,-1] / factor, p0_burn_in_save[:,-4] / factor, p0_burn_in_save[:,-8] / factor

        # Also unscale the initial estimate of the amplitude of the blueshifted Lya peak if the model function had 3 peaks
        p0_burn_in_save[:,0] = p0_burn_in_save[:,0] / factor if model==three_peaks else p0_burn_in_save[:,0]

        # Save the output file containing the best-fit model parameters
        np.savetxt(f'{results}/lya_fits/{slit_id}/{slit_id}_mc_sim_burn_in_lya_best_fit_model_parameters.txt', p0_burn_in_save, header=header, delimiter=' ', encoding='utf-8')

        # Compute the median best-fit model parameters of the burn-in Monte Carlo simulation
        p0_burn_in = np.median(np.array(parameters_burn_in, dtype=np.float64).T, axis=1)

        # Establish empty lists to append the results of the Monte Carlo simulation to
        mc_spectra = []
        parameters = []
        measurements = []

        # For each iteration in the Monte Carlo simulation
        for j in range(10000):

            # Draw a randomly sampled spectrum from the original observation, assuming 
            # that the observed flux densities and associated uncertainties correspond 
            # to the mean and standard deviation of Gaussian distributions, respectively
            f_mc = np.random.normal(f, n)

            # Compute the local continuum as the median flux density between 1221 - 1225 Å in the rest frame
            c = np.median(f_mc[cntm_mask])

            # Compute the rest-frame equivalent width of the Lya profile
            ew = -1 * np.trapz(1 - (f_mc / c)[lya_profile_mask], w[lya_profile_mask])

            # Compute the central escape fraction of the Lya profile
            f_cen = np.trapz(f_mc[v100_mask], v[v100_mask]) / np.trapz(f_mc[v1000_mask], v[v1000_mask]) * 100

            # Compute the luminosity of the Lya profile, placing the data back into the observed frame, removing the scaling factor, and 
            # correcting for the magnification
            l = 4 * np.pi * np.trapz((f_mc[lya_profile_mask] - c) / (1 + z) / mag / factor, (w * (1 + z))[lya_profile_mask]) * l_dist**2

            # Directly fit the Lya profile with the assigned model function and initial parameter estimates and bounds
            p, _ = curve_fit(model, v[(v >= -500) & (f_mc >= c)], f_mc[(v >= -500) & (f_mc >= c)], p0=p0_burn_in, bounds=bounds, maxfev=1e6) 

            # Compute the ratio between the 'minimum' flux density between the redshifted and blueshifted Lya peaks 
            # (really taken as the amplitude of the central Lya peak; see paper for justifying details) and the local continuum
            ratio = p[-4] / p[-1]

            # Append the randomly sampled spectrum, best-fit model parameters, and Lya measurements to the aggregate lists
            mc_spectra.append([*f_mc])
            parameters.append([*p])
            measurements.append([ratio, ew, f_cen, l])

        # Convert all of the result lists into arrays for easier handling
        #p0 = np.array(p0, dtype=np.float64)
        mc_spectra = np.array(mc_spectra, dtype=np.float64)
        parameters = np.array(parameters, dtype=np.float64)
        measurements = np.array(measurements, dtype=np.float64)

        # Unscale the affected best-fit model parameters by the artifical scaling factor
        p0_burn_in[-1], p0_burn_in[-4], p0_burn_in[-8] = p0_burn_in[-1] / factor, p0_burn_in[-4] / factor, p0_burn_in[-8] / factor

        # Also unscale the amplitude of the blueshifted Lya peak if the model function had 3 peaks
        p0_burn_in[0] = p0_burn_in[0] / factor if model==three_peaks else p0_burn_in[0]

        # Unscale the affected best-fit model parameters by the artifical scaling factor
        parameters[:,-1], parameters[:,-4], parameters[:,-8] = parameters[:,-1] / factor, parameters[:,-4] / factor, parameters[:,-8] / factor

        # Also unscale the amplitude of the blueshifted Lya peak if the model function had 3 peaks
        parameters[:,0] = parameters[:,0] / factor if model==three_peaks else parameters[:,0]

        # Replace the Lya luminosity measurements with NaNs if the slit is one of 
        # the stacked spectra, since they do not have physical flux densities
        measurements[:,-1] = np.nan if 'L' in slit_id else measurements[:,-1]

        # Randomly sample spectral resolutions for the spectrum from a Gaussian distribution, 
        # matching the number of iterations of the Monte Carlo simulation, and assuming that 
        # the reported measurement and uncertainty correspond to the mean and standard deviation 
        # of the Gaussian distribution. This is necessary to fold in the uncertainty in the 
        # spectral resolution into the measured FWHMs of the Lya peaks, since they are corrected
        # for instrumental resolution
        Rs = np.random.normal(R, R_error, len(parameters))

        # Calculate the FWHMs of the central Lya peak, correcting for the instrumental resolution
        fwhms_c = 2 * np.sqrt(2 * np.log(2)) * parameters.T[-2]
        fwhms_c = np.sqrt((np.array(fwhms_c))**2 - (Rs)**2)

        fwhms_c = np.where(np.isnan(fwhms_c), Rs, fwhms_c)

        # Calculate the FWHMs and locations of the maxima of the redshifted Lya peak
        fwhms_r, locs_r = compute_fwhm_and_peak(parameters[:, -8:-4], Rs)

        # Insert the central and redshifted Lya peak FWHMs into the Lya measurements array
        measurements = np.insert(measurements, 0, fwhms_r, axis=1)
        measurements = np.insert(measurements, 0, fwhms_c, axis=1)

        # Instantiate the Lya peak separations and blueshifted Lya peak FWHMs as NaNs. This is
        # done to keep the output Lya measurements the same shape for all the spectra, even if
        # they do not have a blueshifted Lya peak necessary to measure the Lya peak separation
        # and blueshifted Lya peak FWHM. If the spectrum does have a third peak, these NaNs
        # will be replaced by actual measurements in the following if statement
        v_sep, fwhms_b = np.empty(len(mc_spectra)), np.empty(len(mc_spectra))
        v_sep[:], fwhms_b[:] = np.nan, np.nan

        # If the spectrum has three Lya peaks
        if slit_id in ['L', 'M0', 'M2', 'M3', 'M7', 'M8', 'M9']:
            
            # Calculate the FWHMs and locations of the maxima of the blueshifted Lya peak
            fwhms_b, locs_b = compute_fwhm_and_peak(parameters[:, 0:4], Rs)

            # Calculate the Lya peak separations
            v_sep = np.abs(locs_r - locs_b)

        # Insert the Lya peak separations and blueshifted Lya peak FWHMs into the Lya measurements array
        measurements = np.insert(measurements, 0, [v_sep, fwhms_b], axis=1)

        # Insert the randomly sampled spectral resolutions into the Lya measurements array, since they are 
        # necessary to measure the instrument-corrected FWHMs
        measurements = np.insert(measurements, 0, 299792.458 / Rs, axis=1)

        # If the results folder for this slit ID doesn't exist yet, make it
        if not os.path.isdir(f'{results}/lya_fits/{slit_id}'):
            os.makedirs(f'{results}/lya_fits/{slit_id}')

        # Set the header for the output file containing the randomly sampled Lya spectra
        header = f'Randomly sampled Lyα spectra of the Monte Carlo simulation of {f"slit {slit_id}" if "M" in slit_id else slit_id}\n' \
            + f'Created by lya.ipynb on {datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")}.\n' \
            + '\n' \
            + f'Number of iterations of this Monte Carlo simulation: {len(mc_spectra)}\n' \
            + '\n' \
            + f'Columns, from left to right: peculiar velocity relative to Lyα (km/s), flux density uncertainty ({flux_density_unit}), and randomly sampled flux densities of each iteration of the Monte Carlo simulation ({flux_density_unit})\n'

        # Save the output file containing the randomly sampled Lya spectra
        np.savetxt(f'{results}/lya_fits/{slit_id}/{slit_id}_mc_sim_lya_spectra.txt', np.array([v, n / factor, *(mc_spectra / factor)]).T, header=header, delimiter=' ', encoding='utf-8')    

        # Set the header for the output file containing the best-fit model parameters
        header = f'Best-fit model parameters of the Lyα fits of the Monte Carlo simulation of {f"slit {slit_id}" if "M" in slit_id else slit_id}.\n' \
            + f'Created by lya.ipynb on {datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")}.\n' \
            + '\n' \
            + f'Initial parameters from the burn-in Monte Carlo simulation: {p0_burn_in}\n' \
            + f'Number of iterations of this Monte Carlo simulation: {len(mc_spectra)}\n' \
            + '\n' \
            + f'Columns, from left to right: {model_parameter_labels}\n'

        # Save the output file containing the best-fit model parameters
        np.savetxt(f'{results}/lya_fits/{slit_id}/{slit_id}_mc_sim_lya_best_fit_model_parameters.txt', parameters, header=header, delimiter=' ', encoding='utf-8')

        # Set the header for the output file containing the Lya measurements
        header = f'Measurements of the Lyα profiles from the best-fit curves of the Monte Carlo simulation of {f"slit {slit_id}" if "M" in slit_id else slit_id}\n' \
            + f'Created by lya.ipynb on {datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")}.\n' \
            + '\n' \
            + f'Number of iterations of this Monte Carlo simulation: {len(mc_spectra)}\n' \
            + '\n' \
            + 'Columns, from left to right: spectral resolution (R), Lyα peak separation (km/s), blueshifted Lyα peak FWHM (km/s), central Lyα peak FWHM (km/s), redshifted Lyα peak FWHM (km/s),\n' \
            + 'ratio between the \'minimum\' flux density between the redshifted and blueshifted Lyα peaks and the continuum flux density (consult the paper for more details),\n' \
            + 'rest-frame Lyα equivalent width (Å), central fraction of Lyα flux (%), and Lyα luminosity (erg/s)\n'

        # Save the output file containing the Lya measurements
        np.savetxt(f'{results}/lya_fits/{slit_id}/{slit_id}_mc_sim_lya_measurements.txt', measurements, header=header, delimiter=' ', encoding='utf-8')

def correlate():

    '''
    Measure the statistical correlations between the Lya measurements
    '''

    # Slit ID dictionary containing, from left to right: file name containing the data, redshift, 
    slits = {
        'NL' : [],
        'L' : [],
        'M5' : [],
        'M4' : [],
        'M6' : [],
        'M3' : [],
        'M0' : [],
        'M2' : [],
        'M7' : [],
        'M8' : [],
        'M9' : []
    }

    # Establish directories
    home = os.getcwd()
    results = f'{home}/results'

    # Calculate the number of iterations of the main Monte Carlo simulation
    iters = len(np.loadtxt(f'{results}/lya_fits/NL/NL_mc_sim_lya_measurements.txt', usecols=(0)))

    # Get the LyC escape fraction measurements and uncertainties from the results file
    f_escs, n_f_escs = np.loadtxt(f'{results}/f_esc_lyc_measurements.txt', delimiter=' ', usecols=(4,5), unpack=True)
    f_escs, n_f_escs = np.insert(f_escs, 0, [np.nan, np.nan]), np.insert(n_f_escs, 0, [np.nan, np.nan])

    # Make an empty list that will contain the statistical correlation results
    corr_coefs = []

    # For each Lya parameter
    for i in range(8):

        # For each unique combination with another Lya parameter not covered by the loop yet
        for j in range(8 - (i + 1)):

            # Make an empty list that will store the aggregate measurements of the parameter pair from all the spectra
            measurements = []

            # For each slit ID
            for k, slit_id in enumerate(slits):

                # If the slit ID is M0 and the Lya luminosity is one of the measurements of the pair
                if slit_id == 'M0' and i+j+2 == 8:

                    # Get the two measurements. Assign the Lya luminosity values as NaNs since they are unphysical for slit M0 due to poor fluxing
                    x, y = np.loadtxt(f'{results}/lya_fits/{slit_id}/{slit_id}_mc_sim_lya_measurements.txt', usecols=(i+1)), np.empty(iters) * np.nan

                    # Rearrange the measurements into an acceptable format
                    measurements.append([[x[i],y[i]] for i in range(len(x))])

                else:

                    # Append the measurements of the parameter pair to the measurement list
                    measurements.append(np.loadtxt(f'{results}/lya_fits/{slit_id}/{slit_id}_mc_sim_lya_measurements.txt', usecols=(i+1, i+j+2)))

            # Make the measurements list into an array for convenience and easier handling
            measurements = np.array(measurements)

            # Make an empty list that will store the aggregate statistical correlations between the measurement pairs
            measurement_pair_corr_coefs = []
            
            #for k, _ in enumerate(measurements):
            # For each iteration in the Monte Carlo simulation
            for k in range(iters):

                # Get all of the measurements of the given measurement pair for that iteration
                pairs = measurements[:,k,:]

                # Drop any pairs that contain NaNs (i.e., where that measurement wasn't applicable)
                pairs = pairs[~np.any(np.isnan(pairs), axis=1)]

                # Take the transpose of the pairs to separate the measurements into an 'x' and 'y' column
                pairs = pairs.T

                # Compute the Pearson and Kendall correlation coefficients
                pearson_corr_coef = np.corrcoef(pairs[0], pairs[1])[0,1]
                kendall_corr_coef = kendalltau(pairs[0], pairs[1], variant='b').statistic

                # Append the correlation coefficients to the aggregate list
                measurement_pair_corr_coefs.append([pearson_corr_coef, kendall_corr_coef])

            # Append all the statistical correlation measurements for the parameter pair to the aggregate list
            corr_coefs.append(measurement_pair_corr_coefs)        

        # Make an empty list that will store the randomly sampled measurements of the LyC escape fractions
        f_esc_lyc = []

        # For each spectrum
        for j, _ in enumerate(slits):

            # Append the randomly sampled measurements (following a Gaussian distribution
            # and assuming that the measured value and associated uncertainty correspond 
            # to the mean and standard deviation of the distribution) of the LyC escape
            # fraction of the slit to the aggregate list
            f_esc_lyc.append(np.random.normal(f_escs[j], n_f_escs[j], iters))

        # Convert the list to an array and transpose it
        f_esc_lyc = np.array(f_esc_lyc).T

        # Make an empty list that will store the aggregate statistical correlations between the measurement pairs
        measurement_pair_corr_coefs = []

        # For each iteration of the Monte Carlo simulation 
        for j in range(iters):

            # Make an array of the measurements organized into pairs by source spectrum
            pairs = np.array([measurements[:,k,0], f_esc_lyc[j]]).T

            # Remove any pairs that contain any NaNs (i.e., where that measurement wasn't applicable)
            pairs = pairs[~np.any(np.isnan(pairs), axis=1)].T

            # Calculate the Pearson and Kendall correlation coefficients
            pearson_corr_coef = np.corrcoef(pairs[0], pairs[1])[0,1]
            kendall_corr_coef = kendalltau(pairs[0], pairs[1], variant='b').statistic

            # Append the correlation coefficients to the aggregate list
            measurement_pair_corr_coefs.append([pearson_corr_coef, kendall_corr_coef])
 
        # Append all the statistical correlation measurements for the parameter pair to the aggregate list
        corr_coefs.append(measurement_pair_corr_coefs)

    # Convert the list of correlation coefficient measurements to an array for easier handling
    corr_coefs = np.array(corr_coefs)

    # Flatten the array so that each set of measurements of a type of correlation coefficient
    # for a parameter pair becomes its own column
    corr_coefs = corr_coefs.swapaxes(1, 2)
    corr_coefs = corr_coefs.reshape(-1, corr_coefs.shape[-1])

    # Transpose the data so that the correlation measurements of a parameter pair are in columns
    corr_coefs = corr_coefs.T

    # Set the header of the output file of the statistical correlations between the measurement pairs
    header = 'Statistical correlations measured between the Lyα measurements each iteration of the Monte Carlo simulation.\n' \
        + f'Created by lya.ipynb on {datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")}.\n' \
        + '\n' \
        + f'Number of iterations of the Monte Carlo simulation: {iters}\n' \
        + '\n' \
        + 'Columns, from left to right: each consecutive pair of columns, starting from the first two columns, are the Pearson \n' \
        + 'correlation coefficients (first column of the pair) and type \'b\' Kendall correlation coefficient (second column \n' \
        + 'of the pair) measured for the following order of Lyα parameter pairs from the Monte Carlo simulation: \n' \
        + '\n' \
        + 'Lyα peak separation - (blueshifted Lyα peak FWHM, central Lyα peak FWHM, redshifted Lyα peak FWHM, ratio between the \'minimum\' flux density between the redshifted and blueshifted Lyα peaks and the continuum flux density, rest-frame Lyα equivalent width, central fraction of Lyα flux, Lyα luminosity), \n' \
        + 'blueshifted Lyα peak FWHM - (central Lyα peak FWHM, redshifted Lyα peak FWHM, ratio between the \'minimum\' flux density between the redshifted and blueshifted Lyα peaks and the continuum flux density, rest-frame Lyα equivalent width, central fraction of Lyα flux, Lyα luminosity), \n' \
        + 'central Lyα peak FWHM - (redshifted Lyα peak FWHM, ratio between the \'minimum\' flux density between the redshifted and blueshifted Lyα peaks and the continuum flux density, rest-frame Lyα equivalent width, central fraction of Lyα flux, Lyα luminosity), \n' \
        + 'redshifted Lyα peak FWHM - (ratio between the \'minimum\' flux density between the redshifted and blueshifted Lyα peaks and the continuum flux density, rest-frame Lyα equivalent width, central fraction of Lyα flux, Lyα luminosity), \n' \
        + 'ratio between the \'minimum\' flux density between the redshifted and blueshifted Lyα peaks and the continuum flux density - (rest-frame Lyα equivalent width, central fraction of Lyα flux, Lyα luminosity), \n' \
        + 'rest-frame Lyα equivalent width - (central fraction of Lyα flux, Lyα luminosity), and\n' \
        + 'central fraction of Lyα flux - Lyα luminosity\n'

    # Save the output file containing the statistical correlations between the measurement pairs
    np.savetxt(f'{results}/lya_fits/mc_sim_lya_measurements_statistical_correlations.txt', corr_coefs, header=header, delimiter='\t', encoding='utf-8')

def plot():

    '''
    Makes various Lya figures: the stacked and individual Lya profiles and Lya measurements corner plot
    '''

    def extract_data(file):

        '''
        Extract spectrum from the .txt file

        Parameters:
            file : str
                Name of the file

        Returns:
            w : numpy.ndarray
                Observed wavelength bins
            f : numpy.ndarray
                Observed flux densities
            n : numpy.ndarray
                Observed Gaussian standard deviation of observed flux densities
        '''

        # Retrieve the data columns
        w, f, n = np.loadtxt(file, delimiter='\t', comments=('#', 'w'), usecols=(0,1,2), unpack=True)
    
        # If the file is not the stacked leaker spectrum. This step is necessary because 
        if 'leaker' not in file:

            # Remove bins of extreme outliers
            w = w[f < 1e-20]
            n = n[f < 1e-20]
            f = f[f < 1e-20]

            # Convert from units of erg/s/cm^2/Hz to erg/s/cm^2/Å
            f = f * 2.998e18 / np.square(w)
            n = n * 2.998e18 / np.square(w)

        return w, f, n

    def two_peaks(x, amp_r, cen_r, width_r, skew_r, amp_c, cen_c, width_c, cntm):

        return amp_r * np.exp(-((x - cen_r) / width_r)**2 / 2) * (1 + erf(skew_r * ((x - cen_r) / width_r) / np.sqrt(2))), \
            amp_c * np.exp(-((x - cen_c) / width_c)**2 / 2), \
            cntm

    def three_peaks(x, amp_b, cen_b, width_b, skew_b, amp_r, cen_r, width_r, skew_r, amp_c, cen_c, width_c, cntm):

        return amp_b * np.exp(-((x - cen_b) / width_b)**2 / 2) * (1 + erf(skew_b * ((x - cen_b) / width_b) / np.sqrt(2))),  \
            amp_r * np.exp(-((x - cen_r) / width_r)**2 / 2) * (1 + erf(skew_r * ((x - cen_r) / width_r) / np.sqrt(2))), \
            amp_c * np.exp(-((x - cen_c) / width_c)**2 / 2), \
            cntm

    # Slit dictionary containing, for each slit ID, from left to right: the file name containing the spectrum,
    # redshift, magnification, peculiar velocity range used to constrain the central Lya peak for purpose of 
    slits = {
        'NL' : ['sunburst_arc_nonleaker_stack_mage.txt', 0, 1, '#DC3220', two_peaks, '#DC3220', 's', 8],
        'L' : ['sunburst_arc_leaker_stack_mage.txt', 0, 1, '#005AB5', three_peaks, '#005AB5', 'o', 8],
        'M5' : ['sunburst_M-5-comb1_MWdr.txt', 2.37086, 51, '#DC3220', two_peaks, 'none', 's', 8],
        'M4' : ['sunburst_M-4-comb1_MWdr.txt', 2.37073, 14.6, '#DC3220', two_peaks, 'none', 's', 8],
        'M6' : ['sunburst_M-6-comb1_MWdr.txt', 2.37021, 147, '#DC3220', two_peaks, 'none', 's', 8],
        'M3' : ['sunburst_M-3-comb1_MWdr.txt', 2.37025, 36, '#D35FB7', three_peaks, 'none', '*', 12],
        'M0' : ['sunburst_M-0-comb1_MWdr.txt', 2.37014, 10, '#005AB5', three_peaks, 'none', 'o', 8],
        'M2' : ['sunburst_M-2-comb1_MWdr.txt', 2.37017, 32, '#005AB5', three_peaks, 'none', 'o', 8],
        'M7' : ['sunburst_M-7-comb1_MWdr.txt', 2.37044, 35, '#005AB5', three_peaks, 'none', 'o', 8],
        'M8' : ['sunburst_M-8-comb1_MWdr.txt', 2.37024, 29, '#005AB5', three_peaks, 'none', 'o', 8],
        'M9' : ['sunburst_M-9-comb1_MWdr.txt', 2.37030, 31, '#005AB5', three_peaks, 'none', 'o', 8]
    }

    # Array containing the padding between the plot and the row unit labels
    labelpads = np.array([27,20,27,20,20,20,20,20], dtype=int)

    # Establish common directories
    home = os.getcwd()
    data = f'{home}/data'
    figs = f'{home}/figs'
    results = f'{home}/results'

    f_escs, n_f_escs = np.loadtxt(f'{results}/f_esc_lyc_measurements.txt', delimiter=' ', usecols=(4,5), unpack=True)
    f_escs, n_f_escs = np.insert(f_escs, 0, [np.nan, np.nan]), np.insert(n_f_escs, 0, [np.nan, np.nan])

    # Make the figure showing the individual Lya profiles

    # Create the figure showing the Lya profiles and best-fit curves
    fig_lya, ax_lya = plt.subplots(3,3, figsize=(12,12), sharex=True, constrained_layout=True)
    
    # Flatten the axes array for easier iteration
    ax_lya_array = np.array(ax_lya).reshape(-1)

    # Multiplicative scaling factor of the data
    factor = 1e17

    # For each slit ID apart from the first two (the stacked spectra)
    for i, slit_id in enumerate(list(slits.keys())[2:]):

        # Get the redshift, color to plot the Lya profile curve, and model function 
        # of the spectrum from the slit dictionary
        z, mag, c, model = slits[slit_id][1], slits[slit_id][2], slits[slit_id][3], slits[slit_id][4]

        # Get the data
        w, f, n = extract_data(f'{data}/mage/{slits[slit_id][0]}')

        # Convert the data to the rest frame
        w, f, n = w / (1 + z), f * (1 + z) / mag, n * (1 + z) / mag

        # Create the Lya peculiar velocities
        v = 299792.458 * (w / 1215.67 - 1)

        # Create a mask to apply to the data, masking between +/- 1100 km/s
        mask = (v >= -1100) & (v <= 1100)

        # Apply the mask to the data
        v, f, n = v[mask], f[mask], n[mask]

        # Get the median best-fit parameters of the model function
        lya_best_fit_model_params = np.median(np.loadtxt(f'{results}/lya_fits/{slit_id}/{slit_id}_mc_sim_lya_best_fit_model_parameters.txt', 
            delimiter=' '), axis=0)

        # Add shading to indicate the 2σ uncertainties of the data
        ax_lya_array[i].fill_between(v, (f - 2 * n) * factor, (f + 2 * n) * factor, step='mid', facecolor=c, alpha=0.3)

        # Plot the data
        ax_lya_array[i].plot(v, f * factor, c=c, ds='steps-mid')

        # Get the best-fit curves of the individual peaks and the continuum level
        best_fit_model_comps = model(v, *lya_best_fit_model_params)

        # Set a variable for the best-fit continuum level from the best-fit curves
        cntm = best_fit_model_comps[-1]

        # For each best-fit curves of the individual peaks
        for j, curve in enumerate(best_fit_model_comps[:-1]):

            # Plot tbest-fit curve to the individual peak
            ax_lya_array[i].plot(v, (curve + cntm) * factor / mag, c='black', ls='dotted')

        # Plot the total best-fit curve to the profile
        ax_lya_array[i].plot(v, (np.sum(best_fit_model_comps[:-1], axis=0) + cntm) * factor / mag, c='black', ls='dashed')

        # Add a label to the corner of the plot indicating the slit ID
        at = AnchoredText(slit_id, frameon=False, loc='upper right', prop=dict(fontsize='x-large'))       
        ax_lya_array[i].add_artist(at)

        # Set the x- and y-axis limits of the plot
        ax_lya_array[i].set_xlim(-1000,1000)
        ax_lya_array[i].set_ylim(bottom=0)

        # Set the locations and labels of the x-axis ticks
        ax_lya_array[i].set_xticks([-1000,-500,0,500,1000], 
            ['$-$1000','$-$500','0','$+$500','$+$1000'])

        # Add ticks to each side of the plot, and point them inward
        ax_lya_array[i].tick_params(left=True, bottom=True, right=True, top=True, direction='in')

        # Set the plot to be square
        ax_lya_array[i].set_aspect(1 / ax_lya_array[i].get_data_ratio(), adjustable='box')

    # Add x- and y-axis labels to the figure
    fig_lya.supylabel(f'Flux density ($10^{{-{abs(floor(log10(abs(factor))))}}}$ erg s$^{{-1}}$ cm$^{{-2}}$ Å$^{{-1}}$)', fontsize='large')
    ax_lya[2,1].set_xlabel('Velocity (km s$^{-1}$)', fontsize='large')

    # Save the figure
    fig_lya.savefig(f'{figs}/lya_spectra/lya_fits.pdf', bbox_inches='tight')

    # Make the figure showing the stacked Lya spectra

    # Instantiate the figure showing the stacked Lya spectra and best-fit models
    fig_lya_stack, ax_lya_stack = plt.subplots(3,1, figsize=(3,9), sharex=True)

    # For the first two slit IDs (the stacked spectra)
    for i, slit_id in enumerate(list(slits.keys())[:2]):

        # Get the redshift, color to plot the Lya profile curve, and model function 
        # of the spectrum from the slit dictionary
        z, c, model = slits[slit_id][1], slits[slit_id][3], slits[slit_id][4]

        # Get the data
        w, f, n = extract_data(f'{data}/mage/{slits[slit_id][0]}')

        # Convert the data to the rest frame
        w, f, n = w / (1 + z), f * (1 + z), n * (1 + z)

        # Create the Lya peculiar velocities
        v = 299792.458 * (w / 1215.67 - 1)

        # Create a mask to apply to the data, masking between +/- 1100 km/s
        mask = (v >= -1100) & (v <= 1100)

        # Apply the mask to the data
        v, f, n = v[mask], f[mask], n[mask]

        # Get the median best-fit parameters of the model function
        lya_best_fit_model_params = np.median(np.loadtxt(f'{results}/lya_fits/{slit_id}/{slit_id}_mc_sim_lya_best_fit_model_parameters.txt', 
            delimiter=' '), axis=0)

        # Get the best-fit curves of the individual peaks and the continuum level
        best_fit_model_comps = model(v, *lya_best_fit_model_params)

        # Add shading to indicate the 3σ uncertainties of the data
        ax_lya_stack[0].fill_between(v, f - 3 * n, f + 3 * n, step='mid', facecolor=c, alpha=0.3)
        ax_lya_stack[i+1].fill_between(v, f - 3 * n, f + 3 * n, step='mid', facecolor=c, alpha=0.3)

        # Plot the spectrum on the subplot of the shared profile and a standalone subplot for the
        # profile
        ax_lya_stack[0].plot(v, f, color=c, ds='steps-mid')
        ax_lya_stack[i+1].plot(v, f, color=c, ds='steps-mid')

        # Set a variable for the best-fit continuum level from the best-fit curves
        cntm = best_fit_model_comps[-1]

        # For each best-fit curves of the individual peaks
        for j, curve in enumerate(best_fit_model_comps[:-1]):

            # Plot tbest-fit curve to the individual peak
            ax_lya_stack[i+1].plot(v, (curve + cntm), c='black', ls='dotted')

        # Plot the total fit against the standalone spectrum
        ax_lya_stack[i+1].plot(v, np.sum(best_fit_model_comps[:-1], axis=0) + best_fit_model_comps[-1], color='black', ls='dashed')

        # Add an annotation with the slit ID to a corner of the standalone spectrum subplot
        at = AnchoredText(slit_id, frameon=False, loc='upper right', prop=dict(fontsize='large'))
        ax_lya_stack[i+1].add_artist(at)

        # Set the x- and y-axis limits of the standalone spectrum subplot
        ax_lya_stack[i+1].set_xlim(-1000,1000)
        ax_lya_stack[i+1].set_ylim(bottom=0)

        # Add inward-facing ticks to all sides of the standalone spectrum subplot
        ax_lya_stack[i+1].tick_params(left=True, bottom=True, right=True, top=True, direction='in')

    # Add x- and y-axis labels to the figure
    ax_lya_stack[-1].set_xlabel('Velocity (km s$^{-1}$)', fontsize='large')
    ax_lya_stack[1].set_ylabel('Flux density (normalized)', fontsize='large')

    # Set the y-axis limits of the shared subplot
    ax_lya_stack[0].set_ylim(bottom=0)

    # Set the y-axis limits of the stacked non-LyC-leaking apertures subplot
    ax_lya_stack[1].set_ylim(top=12)

    # Set the x-axis tick locations and labels of the shared subplot
    ax_lya_stack[0].set_xticks([-1000,-500,0,500,1000], ['$-$1000','$-$500','$0$','$+$500','$+$1000'])

    # Set the y-axis tick locations and labels of the stacked nonleaker spectrum subplot
    ax_lya_stack[1].set_yticks([0,3,6,9], ['0','3','6','9'])

    # Add inward-facing ticks to each side of the top subplot
    ax_lya_stack[0].tick_params(left=True, bottom=True, right=True, top=True, direction='in')

    # Eliminate space between the subplots
    fig_lya_stack.subplots_adjust(hspace=0, wspace=0)

    # Save the figure as a PDF
    fig_lya_stack.savefig(f'{figs}/lya_spectra/lya_fits_stack.pdf', bbox_inches='tight')

    # Make the corner plot figure of the Lya measurements and LyC escape fraction

    # Instantiate the figure
    fig_corner, ax_corner = plt.subplots(8,8, figsize=(16,16), sharex='col', sharey='row', constrained_layout=True)
    fig_corner.tight_layout()

    # Make arrays with the titles of the columns and rows
    col_titles = np.array([r'$v_{\rm{sep}}$', 'FWHM (blue)', 'FWHM (center)', 'FWHM (red)', r'$f_{\rm{min}}/f_{\rm{cont}}$', 'EW', r'$f_{\rm{cen}}$', 'Luminosity'], dtype=str)
    row_titles = np.array(['FWHM (blue)', 'FWHM (center)', 'FWHM (red)', r'$f_{\rm{min}}/f_{\rm{cont}}$', 'EW', r'$f_{\rm{cen}}$', 'Luminosity', r'$f_{\rm{esc}}^{\rm{LyC}}$'], dtype=str)

    # Make arrays with the unit labels of the columns and rows
    col_labels = np.array(['(km s$^{-1}$)', '(km s$^{-1}$)', '(km s$^{-1}$)', '(km s$^{-1}$)', '', '(Å)', '(%)', '(10$^{41}$ erg s$^{-1}$)'], dtype=str)
    row_labels = np.array(['(km s$^{-1}$)', '(km s$^{-1}$)', '(km s$^{-1}$)', '', '(Å)', '(%)', '(10$^{41}$ erg s$^{-1}$)', '(%)'], dtype=str)

    # Open the .txt file containing the measurement of the statistical correlations between the different Lya measurements
    file = open(f'{results}/tables/lya_measurements_statistical_correlations_table.tex')

    # Get just the tabulated values, split apart by file line
    lines = file.readlines()[10:46]

    # Make an array with the numbered location codes of the location to annotate 
    # the Pearson correlation coefficient of the measurements on each plot
    locs = np.array([
        3,
        3,4,
        2,2,2,
        3,3,6,7,
        3,4,6,7,4,
        3,4,6,7,2,2,
        4,4,2,4,2,2,2,
        4,1,2,1,2,2,2,2], 
        dtype=int)

    # For each row of subplots in the figure except the last row (the LyC escape fractions row)
    for i, row in enumerate(ax_corner[:-1]):

        # Set the column title of the last subplot in the row
        ax_corner[i,i].set_title(col_titles[i], fontsize='large')

        # Set the row title
        ax_corner[i,0].set_ylabel(row_titles[i], fontsize='large')

        # Set the column unit label of the last column in the row
        ax_corner[-1,i].set_xlabel(col_labels[i], fontsize='large')

        # Make a twin set of axes of the last subplot in the row
        subplot_twin = ax_corner[i,i].twinx()

        # Set the row unit label of the last subplot in the row
        subplot_twin.set_ylabel(row_labels[i], fontsize='large', rotation=-90, labelpad=labelpads[i])

        # Make an empty set of ticks and labels on the twin axes (otherwise, 
        # automatic ticks in the axes coordinate system will appear)
        subplot_twin.set_yticks([0.5], ['   '])
        subplot_twin.tick_params(left=False, bottom=False, right=False, top=False)

        # For each subplot in the row
        for j, subplot in enumerate(row):

            # If the subplot is above the main diagonal
            if j > i:

                # Hide the subplot
                subplot.axis('off')

            else:

                # For each slit ID
                for k, slit_id in enumerate(slits):

                    # If the row is the Lya luminosity and the slit ID is M0, pass, since the Lya luminosity 
                    # of slit M0 is unphysically large due to a poor fluxing of the observations
                    if i == 6 and slit_id == 'M0':
                        pass

                    else:

                        # Get the color, marker, and marker size properties of the slit ID
                        ec, fc, marker, size = slits[slit_id][3], slits[slit_id][5], slits[slit_id][6], slits[slit_id][7]

                        # Get the measurements of the plotted measurement pair for the given slit ID
                        x, y = np.loadtxt(f'{results}/lya_fits/{slit_id}/{slit_id}_mc_sim_lya_measurements.txt', 
                            delimiter=' ', comments='#', usecols=(j+1,i+2), unpack=True)

                        # Calculate the medians of the measurement distributions
                        medians = [np.median(x), np.median(y)]

                        # Make arrays of the uncertainties of the two measurement distributions
                        x_error = np.array([[medians[0] - np.percentile(x, 16)], [np.percentile(x, 84) - medians[0]]], dtype=np.float64)
                        y_error = np.array([[medians[1] - np.percentile(y, 16)], [np.percentile(y, 84) - medians[1]]], dtype=np.float64)

                        # If the row is the Lya luminosity, rescale the data by a multiplicative factor to prevent an annotated scientific notation label
                        medians, y_error = ([medians[0], medians[1] * 1e-41], y_error * 1e-41) if i==6 else (medians, y_error)

                        # Plot the medians with uncertainties
                        subplot.errorbar(*medians, xerr=x_error, yerr=y_error, lw=1, mec=ec, mfc=fc, ecolor=ec, marker=marker, markersize=size)

                # Hide labels on the left and top sides of the subplot
                subplot.tick_params(labelleft=False, labeltop=False)

                # If the subplot is on the main diagonal
                if i == j:

                    # Add tick labels to the right side of the subplot
                    subplot.tick_params(labelright=True)

                # Add inward-facing ticks to each side of the subplot
                subplot.tick_params(left=True, bottom=True, right=True, top=True, direction='in')

                # Add the Pearson correlation coefficient measurement for the measurement pair to the subplot
                at = AnchoredText(lines[int(i + (7 * (7 + 1) / 2) - ((7 - i) * (7 - i + 1) / 2))].split('& ')[1], loc=locs[int(i * (i + 1) / 2 + j)], frameon=False)
                subplot.add_artist(at)

    # For each subplot in the last row
    for i in range(len(ax_corner[-1])):

        # For each slit ID
        for j, slit_id in enumerate(slits):

            # If the subplot is the last in the row (i.e., the column is Lya luminosity) and the slit ID is M0, 
            # pass, since the Lya luminosity in that instance is unphysically large due to a poor fluxing
            if i == len(ax_corner[0]) - 1 and slit_id == 'M0':
                pass

            else:

                # Get the color, marker, and marker size properties of the slit ID
                ec, fc, marker, size = slits[slit_id][3], slits[slit_id][5], slits[slit_id][6], slits[slit_id][7]

                # Get the measurements of the plotted measurement pair for the given slit ID
                x = np.loadtxt(f'{results}/lya_fits/{slit_id}/{slit_id}_mc_sim_lya_measurements.txt', 
                    delimiter=' ', comments='#', usecols=(i+1), unpack=True)
                y = f_escs[j]

                # Make an array of the x-axis uncertainties
                x_error = np.array([[np.median(x) - np.percentile(x, 16)], [np.percentile(x, 84) - np.median(x)]], dtype=np.float64)

                # Rescale the data if the subplot is the last in the row (i.e., if it is the Lya luminosity column) for 
                # better aesthetics to prevent an automatic scientific notation factor
                x, x_error = (x * 1e-41, x_error * 1e-41) if i==len(ax_corner[0]) - 1 else (x, x_error)

                # Plot the measurements with uncertainties
                ax_corner[-1,i].errorbar(np.median(x), y, xerr=x_error, yerr=n_f_escs[j], lw=1, mec=ec, mfc=fc, ecolor=ec, marker=marker, markersize=size)

        # Add inward-facing ticks to each side of the subplot and tick labels to the bottom of the subplot
        ax_corner[-1,i].tick_params(left=True, bottom=True, right=True, top=True,
            labelleft=False, labelbottom=True, labeltop=False, direction='in')

        # Add the Pearson correlation coefficient measurement for the measurement pair to the subplot
        at = AnchoredText(lines[int(i + (7 * (7 + 1) / 2) - ((7 - i) * (7 - i + 1) / 2))].split('& ')[1], loc=locs[int(7 * (7 + 1) / 2 + i)], frameon=False)
        ax_corner[-1,i].add_artist(at)

    # Set the column title of the bottom right corner subplot
    ax_corner[-1,-1].set_title(col_titles[-1], fontsize='large')

    # Set the row title of the bottom row
    ax_corner[-1,0].set_ylabel(row_titles[-1], fontsize='large')

    # Add the column unit label of the last column
    ax_corner[-1,-1].set_xlabel(col_labels[-1], fontsize='large')

    # Make a twin axes of the bottom right subplot to add the y-axis label
    ax_twin = ax_corner[-1,-1].twinx()

    # Add the row unit label 
    ax_twin.set_ylabel(row_labels[-1], fontsize='large', rotation=-90, labelpad=labelpads[-1])

    # Make an empty set of ticks and labels on the twin axes (otherwise, 
    # automatic ticks in the axes coordinate system will appear)
    ax_twin.set_yticks([0.5], ['   '])
    ax_twin.tick_params(left=False, bottom=False, right=False, top=False)

    # Add inward-facing ticks to all sides and tick labels to the bottom and right sides of 
    # the bottom right corner subplot
    ax_corner[-1,-1].tick_params(left=True, bottom=True, right=True, top=True, 
        labelleft=False, labelbottom=True, labelright=True, labeltop=False, direction='in')

    # Save the figure
    fig_corner.savefig(f'{figs}/lya_measurements_corner.pdf', bbox_inches='tight')

def tabulate():

    '''
    Saves the LaTeX-formatted tables (the Lya measurements, statistical correlations, 
    and model parameters) as .txt files
    '''

    def round_to_uncertainties(median, lower, upper):

        '''
        Round the uncertainties to one significant figure and the median to
        the most precise significant digit of the two uncertainties

        Parameters:

            median : numpy.float64
                Median value of the measurement distribution

            lower : numpy.float64
                Absolute difference between the median and 16th percentile of 
                the measurement distribution

            upper : numpy.float64
                Absolute difference between the median and 84th percentile of
                the measurement distribution

        Returns:

            median : str
                Median value of the measurement distribution, rounded to the most precise
                significant digit of the one significant figure-rounded uncertainties

            lower : str
                Absolute difference between the median and 16th percentile of the 
                measurement distribution, rounded to one significant figure

            upper : str
                Absolute difference between the median and 84th percentile of the 
                measurement distribution, rounded to one significant figure
        '''

        # Round the lower and upper uncertainties to one significant figure
        lower = sigfig.round(lower, sigfigs=1, type=str)
        upper = sigfig.round(upper, sigfigs=1, type=str)

        # Get whichever bound is smaller, in order to round the median to
        # the digit of that bound's significant figure
        ref = min(np.array([lower, upper], dtype=str), key=float)

        # If the smaller bound is less than 1
        if '.' in ref:

            # Round the median to the same digit as the smaller bound's only significant figure
            median = sigfig.round(median, decimals=len(ref.split('.')[1]), type=str)

        # If the smaller bound is greater than 1
        elif '.' not in ref:

            # Round the median to the same digit as the smaller bound's only significant figure
            median = sigfig.round(median, len(str(median).replace('-','').split('.')[0]) - len(ref) + 1, type=str)

        return median, lower, upper
            
    # Dictionary containing the spectral resolutions of the slits
    slits = {
        'NL' : [5400],
        'L' : [5300],
        'M5' : [5500],
        'M4' : [5400],
        'M6' : [5300],
        'M3' : [5500],
        'M0' : [4700],
        'M2' : [5300],
        'M7' : [5200],
        'M8' : [5200],
        'M9' : [5500]
    }

    # Establish common directories
    home = os.getcwd()
    results = f'{home}/results'

    # Make the Lya measurements table    

    # Header of the table containing the Lya measurements
    table = '\\begin{deluxetable*}{cllllllll}[ht!]\n\n' \
        + '\\tablecaption{Ly$\\alpha$ measurements \label{tab:lya_params}}\n\n' \
        + '\\tablehead{\n' \
        + '\t\colhead{Slit} & \colhead{$v_{\\rm{sep}}$} & \colhead{FWHM (blue)} & \colhead{FWHM (center)} & \colhead{FWHM (red)} & \colhead{$f_{\\rm{min}}/f_{\\rm{cont}}$} & \colhead{EW} & \colhead{$f_{\\rm{cen}}$} & \colhead{Luminosity}\n' \
        + '\t\\\\\n' \
        + '\t\colhead{} &\n' \
        + '\t\colhead{[km s$^{-1}$]} &\n' \
        + '\t\colhead{[km s$^{-1}$]} &\n' \
        + '\t\colhead{[km s$^{-1}$]} &\n' \
        + '\t\colhead{[km s$^{-1}$]} &\n' \
        + '\t\colhead{} &\n' \
        + '\t\colhead{[\AA]} &\n' \
        + '\t\colhead{[\%]} &\n' \
        + '\t\colhead{[$10^{41}$ erg s$^{-1}$]}\n' \
        + '}\n\n' \
        + '\startdata\n'

    # For each slit ID
    for i, slit_id in enumerate(slits):

        # Load the Lya measurements of that slit
        slit_lya_measurements = np.loadtxt(f'{results}/lya_fits/{slit_id}/{slit_id}_mc_sim_lya_measurements.txt', delimiter=' ', comments='#').T
        
        # Remove the spectral resolution measurements
        slit_lya_measurements = slit_lya_measurements[1:]

        # Add a separating horizontal line between the stacked spectra, the individual LyC-leaking spectra, and the individual non-LyC-leaking spectra
        table = table + '\hline\n' if slit_id in ['M5', 'M0'] else table + ''

        # Add the slit ID to the table row
        table = table + f'{slit_id} '

        # For each set of Lya measurements for the slit
        for j, measurements in enumerate(slit_lya_measurements):

            # Determine the median and lower and upper uncertainties 
            median = np.percentile(measurements, 50)
            lower = median - np.percentile(measurements, 16)
            upper = np.percentile(measurements, 84) - median

            # Rescale the median and lower and upper measurements if the measurement is the Lya luminosity
            median, lower, upper = (median / 1e41, lower / 1e41, upper / 1e41) if j+1 == len(slit_lya_measurements) else (median, lower, upper)

            # If the measurement was not invalid for the given slit (e.g., Lya peak 
            # separations for profiles missing one of the two peaks would be invalid)
            if not np.isnan(median):

                # Round the uncertainties to one significant figure and the median to match 
                # the most significant digit between the two uncertainties
                median, lower, upper = round_to_uncertainties(median, lower, upper)

                # If the measurement is the central Lya peak FWHM
                if j==2:

                    # Get the spectral resolution of the spectrum in units of km/s
                    R = 299792.458 / slits[slit_id][0]

                    # If the 16th percentile of the measurement distribution is less than
                    # the instrumental FWHM
                    if eval(f'{median} - {lower}') < R:

                        # Report the measurement by listing the 84th percentile of the 
                        # measurement distribution as an upper bound
                        table = table + f'& \lesssim {eval(f"{median} + {upper}")}'

                    else:

                        # Add the formatted measurement to the table
                        table = table + f'& ${median}_{{-{lower}}}^{{+{upper}}}$ '           
                else:

                    # Add the formatted measurement to the table
                    table = table + f'& ${median}_{{-{lower}}}^{{+{upper}}}$ '
            
            else:

                # Add a dash to the table to represent the measurement is invalid
                table = table + '& $---$ '
        
        # Add a LaTeX row break character to the end of the two, unless the row is the
        # last row, and add a line break to the .txt file
        table = table + '\\\\' + '\n' if i != len(slits) - 1 else table + '\n'

    # Add a footer to the table
    table = table + '\\enddata\n\n' \
        + '\\tablecomments{From left to right: slit label, peak separation between the redshifted and blueshifted Ly$\\alpha$ peaks ' \
        + '(km s$^{-1}$), FWHM of the blueshifted, central, and redshifted Ly$\\alpha$ peaks (km s$^{-1}$), respectively, ratio between ' \
        + 'the `minimum\' flux density between the redshifted and blueshifted Ly$\\alpha$ peaks and the local continuum flux density, ' \
        + 'rest-frame Ly$\\alpha$ equivalent width ({\AA}), central escape fraction (\%), and magnification-corrected Ly$\\alpha$ luminosity ' \
        + '(10$^{41}$ erg s$^{-1}$). Because the deconvolved FWHMs of the central Ly$\\alpha$ peaks of slits M4 and M5 were not significantly ' \
        + 'greater than the instrumental line spread function FWHM ($\sim55$ km s$^{-1}$), we quote the 84th percentiles of those measurements ' \
        + 'as an upper bound on the intrinsic FWHM of their central Ly$\\alpha$ peaks.}\n' \
        + '\n' \
        + '\\tablenotetext{a}{Slit M0\'s observation was taken through thin cloud cover that prevented an accurate fluxing, so its ' \
        + 'significantly larger luminosity is not an accurate estimate. We do not include this data point in any figures or when ' \
        + 'estimating any correlations involving the Ly$\\alpha$ luminosity. See Table \\ref{tab:mage_log} for more information about ' \
        + 'the observation.}\n' \
        + '\n' \
        + '\end{deluxetable*}'

    # Save the table
    f = open(f'{results}/tables/lya_measurements_table.tex', 'w', encoding='utf-8')
    f.write(table)
    f.close()

    # Make the statistical correlation measurements table

    # Create the header for the statistical correlation measurements table
    table = '\\begin{deluxetable}{lrr}\n' \
        + '\n' \
        + '\\tablecaption{Correlations between Ly$\\alpha$ and LyC parameters \label{tab:param_correlations}}\n' \
        + '\n' \
        + '\\tablehead{\n' \
        + '\t\colhead{} & \colhead{$r$} & \colhead{$\\tau$}\n' \
        + '}\n' \
        + '\n' \
        + '\startdata\n'

    # Make the array containing the row labels of the table
    row_labels = np.array([
        'FWHM (b) - $v_{\\rm{sep}}$',
        'FWHM (c) - $v_{\\rm{sep}}$',
        'FWHM (r) - $v_{\\rm{sep}}$',
        '$f_{\\rm{min}}/f_{\\rm{cont}} - $v_{\\rm{sep}}$',
        'Ly$\\alpha$ EW - $v_{\\rm{sep}}$',
        '$f_{\\rm{cen}}$ - $v_{\\rm{sep}}$',
        'Ly$\\alpha$ $L$ - $v_{\\rm{sep}}$',
        '$f_{\\rm{esc}}^{\\rm{LyC}}$ - $v_{\\rm{sep}}$',
        'FWHM (c) - FWHM (b)',
        'FWHM (r) - FWHM (b)',
        '$f_{\\rm{min}}/f_{\\rm{cont}} - FWHM (b)',
        'Ly$\\alpha$ EW - FWHM (b)',
        '$f_{\\rm{cen}}$ - FWHM (b)',
        'Ly$\\alpha$ $L$ - FWHM (b)',
        '$f_{\\rm{esc}}^{\\rm{LyC}}$ - FWHM (b)',
        'FWHM (r) - FWHM (c)',
        '$f_{\\rm{min}}/f_{\\rm{cont}} - FWHM (c)',
        'Ly$\\alpha$ EW - FWHM (c)',
        '$f_{\\rm{cen}}$ - FWHM (c)',
        'Ly$\\alpha$ $L$ - FWHM (c)',
        '$f_{\\rm{esc}}^{\\rm{LyC}}$ - FWHM (c)',
        '$f_{\\rm{min}}/f_{\\rm{cont}} - FWHM (r)',
        'Ly$\\alpha$ EW - FWHM (r)',
        '$f_{\\rm{cen}}$ - FWHM (r)',
        'Ly$\\alpha$ $L$ - FWHM (r)',
        '$f_{\\rm{esc}}^{\\rm{LyC}}$ - FWHM (r)',
        'Ly$\\alpha$ EW - f_{\\rm{min}}/f_{\\rm{cont}}',
        '$f_{\\rm{cen}}$ - f_{\\rm{min}}/f_{\\rm{cont}}',
        'Ly$\\alpha$ $L$ - f_{\\rm{min}}/f_{\\rm{cont}}',
        '$f_{\\rm{esc}}^{\\rm{LyC}}$ - f_{\\rm{min}}/f_{\\rm{cont}}',
        '$f_{\\rm{cen}}$ - Ly$\\alpha$ EW',
        'Ly$\\alpha$ $L$ - Ly$\\alpha$ EW',
        '$f_{\\rm{esc}}^{\\rm{LyC}}$ - Ly$\\alpha$ EW',
        'Ly$\\alpha$ $L$ - $f_{\\rm{cen}}',
        '$f_{\\rm{esc}}^{\\rm{LyC}}$ - $f_{\\rm{cen}}',
        '$f_{\\rm{esc}}^{\\rm{LyC}}$ - Ly$\\alpha$ $L$',
    ], dtype=str)

    # Get the statistical correlation measurements
    corr_coefs = np.loadtxt(f'{results}/lya_fits/mc_sim_lya_measurements_statistical_correlations.txt', delimiter='\t', comments='#')

    # For each measurement pair
    for i in range(len(corr_coefs.T) // 2):

        # Add the row label to the row
        table = table + f'{row_labels[i]} '

        # Get the measurements of the two statistical correlation coefficients for the measurement pair
        corr_coef_pair = corr_coefs[:,2*i:2*i+2]

        # Calculate the medians of the two statistical correlation coefficients
        medians = np.median(corr_coef_pair, axis=0)

        # Calculate the lower and upper uncertainties of the two statistical correlation coefficients
        lowers = medians - np.percentile(corr_coef_pair, 16, axis=0)
        uppers = np.percentile(corr_coef_pair, 84, axis=0) - medians

        # For each statistical correlation coefficient type
        for j in range(len(medians)):

            # Round the lower and upper uncertainties to one significant figure and the median 
            # to the most significant digit between the uncertainties
            median, lower, upper = round_to_uncertainties(medians[j], lowers[j], uppers[j])

            # Add the formatted measurement to the table row
            table = table + f'& ${median}_{{-{lower}}}^{{+{upper}}}$ '

        # Add a LaTeX row break character to the end of the row, unless the row is the
        # last row, and add a line break to the .txt file
        table = table + '\\\\\n' if i != len(corr_coefs.T) / 2 - 1 else table + '\n'
    
    # Add a footer to the table
    table = table + '\enddata\n' \
        + '\n' \
        + '\\tablecomments{Statistical correlations between the Ly$\\alpha$ parameters and $f_{\\rm{esc}}^{\\rm{LyC}}$. ' \
        + 'From left to right: the parameter pair, Pearson correlation coefficient $r$, and type `b\' Kendall rank ' \
        + 'correlation coefficient $\\tau$. The minimal number of data points (no more than 11 for any pair of parameters) ' \
        + 'means there are not many unique values of $\\tau$, which causes many of the listed values and uncertainties ' \
        + 'to be similar, or in extreme cases for high SNR parameter measurements, for the 16th and 84th percentiles ' \
        + 'listed to be the same value as the median.}\n' \
        + '\n' \
        + '\\end{deluxetable}'
    
    # Save the table
    f = open(f'{results}/tables/lya_measurements_statistical_correlations_table.tex', 'w', encoding='utf-8')
    f.write(table)
    f.close()

    # Make the table of the best-fit Lya model parameters    

    # Make the header
    table = '\\begin{deluxetable}{lcccc}\n' \
        + '\n' \
        + '\\rotate\n' \
        + '\n' \
        + '\\tablecaption{Ly$\\alpha$ modeling best-fit parameters \label{tab:fit_results}}\n' \
        + '\n' \
        + '\\tablehead{\n' \
        + '\t\colhead{Slit} & \colhead{Blue peak} & \colhead{Red peak} & \colhead{Central peak} & \colhead{Continuum}\n' \
        + '}\n' \
        + '\n' \
        + '\startdata\n'

    # For each slit ID
    for i, slit_id in enumerate(slits):

        # Add a separating horizontal line between the LyC-leaking and non-LyC-leaking spectra
        table = table + '\hline\n' if slit_id == 'M0' else table + ''

        # Add the slit ID to the row
        table = table + f'{slit_id} & '

        # Get the best-fit Lya model parameters of the spectrum
        params = np.loadtxt(f'{results}/lya_fits/{slit_id}/{slit_id}_mc_sim_lya_best_fit_model_parameters.txt', delimiter=' ', comments='#').T

        # If the slit has a triple-peaked Lya profile
        if slit_id in ['L', 'M3', 'M0', 'M2', 'M7', 'M8', 'M9']:

            # For the best-fit parameter distributions of the blueshifted Lya peak
            for j, param in enumerate(params[0:4]):

                # Rescale the parameter distribution if it is the amplitude and the spectrum is 
                # not one of the stacked spectra (because the stacked spectra are normalized)
                param = param * 1e16 if slit_id not in ['NL','L'] and j==0 else param

                # Calculate the median of the measured parameter distribution
                median = np.median(param)

                # Calculate the lower and upper uncertainties of the parameter distribution
                lower, upper = median - np.percentile(param, 16), np.percentile(param, 84) - median

                # Round the lower and upper uncertainties to one significant figure and 
                # the median to the most significant digit between the two uncertainties
                median, lower, upper = round_to_uncertainties(median, lower, upper)

                # Add the formatted parameter measurement to the table row
                table = table + f'${median}_{{-{lower}}}^{{+{upper}}}$ '

                # Add an aesthetic, dividing '/' after the formatted parameter 
                # measurement, unless it is the last parameter in the loop
                table = table + '/ ' if j != len(params[0:4]) - 1 else table

        else:

            # Add an empty set of parameter measurements, since the spectrum's 
            # model did not include a blueshifted Lya peak
            table = table + '$-$ / $-$ / $-$ / $-$ '

        # Add a LaTeX table column break character
        table = table + '& '

        # For the best-fit parameter distributions of the redshifted Lya peak
        for j, param in enumerate(params[-8:-4]):

            # Rescale the parameter distribution if it is the amplitude and the spectrum is 
            # not one of the stacked spectra (because the stacked spectra are normalized)
            param = param * 1e16 if slit_id not in ['NL','L'] and j==0 else param

            # Calculate the median of the measured parameter distribution
            median = np.median(param)

            # Calculate the lower and upper uncertainties of the parameter distribution
            lower, upper = median - np.percentile(param, 16), np.percentile(param, 84) - median

            # Round the lower and upper uncertainties to one significant figure and 
            # the median to the most significant digit between the two uncertainties
            median, lower, upper = round_to_uncertainties(median, lower, upper)

            # Add the formatted parameter measurement to the table row
            table = table + f'${median}_{{-{lower}}}^{{+{upper}}}$ '

            # Add an aesthetic, dividing '/' after the formatted parameter 
            # measurement, unless it is the last parameter in the loop
            table = table + '/ ' if j != len(params[-8:-4]) - 1 else table

        # Add a LaTeX table column break character
        table = table + '& '    

        # For the best-fit parameter distributions of the central Lya peak
        for j, param in enumerate(params[-4:-1]):

            # Rescale the parameter distribution if it is the amplitude and the spectrum is 
            # not one of the stacked spectra (because the stacked spectra are normalized)
            param = param * 1e16 if slit_id not in ['NL','L'] and j==0 else param

            # Calculate the median of the measured parameter distribution
            median = np.median(param)

            # Calculate the lower and upper uncertainties of the parameter distribution
            lower, upper = median - np.percentile(param, 16), np.percentile(param, 84) - median

            # Round the lower and upper uncertainties to one significant figure and 
            # the median to the most significant digit between the two uncertainties
            median, lower, upper = round_to_uncertainties(median, lower, upper)

            # Add the formatted parameter measurement to the table row
            table = table + f'${median}_{{-{lower}}}^{{+{upper}}}$ '

            # Add an aesthetic, dividing '/' after the formatted parameter 
            # measurement, unless it is the last parameter in the loop
            table = table + '/ ' if j != len(params[-4:-1]) - 1 else table

        # Add a final empty measurement to the group of central Lya peak measurements for consistency
        table = table + '/ $-$ '

        # Get the parameter distribution of the local continuum
        param = params[-1]

        # Rescale the parameter distribution if the spectrum is not one of 
        # the stacked spectra (because the stacked spectra are normalized)
        param = param * 1e16 if slit_id not in ['NL','L'] else param

        # Calculate the median of the measured parameter distribution
        median = np.median(param)

        # Calculate the lower and upper uncertainties of the parameter distribution
        lower, upper = median - np.percentile(param, 16), np.percentile(param, 84) - median

        # Round the lower and upper uncertainties to one significant figure and 
        # the median to the most significant digit between the two uncertainties
        median, lower, upper = round_to_uncertainties(median, lower, upper)

        # Add the formatted parameter measurement to the table row
        table = table + f'& ${median}_{{-{lower}}}^{{+{upper}}}$ '

        # Add a LaTeX row break character if the row is not the last, and a line break character to the string
        table = table + '\\\\\n' if i != len(slits) - 1 else table + '\n'

    # Add a footer to the table
    table = table + '\enddata\n' \
        + '\n' \
        + '\\tablecomments{Best-fit Ly$\\alpha$ parameters of the magnification-uncorrected, rest-frame Ly$\\alpha$ profiles. From left to right: slit label, best-fit parameters for the blue peak, red peak, and central peak, presented as $\\alpha$/$\mu$/$\sigma$/$\omega$, as described in \S\,\\ref{sssec:methods_lya_fwhm_vsep}. Apart from the stacked spectra, $\\alpha$ has units of $10^{-16}$ erg s$^{-1}$ cm$^{-2}$ {\AA}$^{-1}$, and $\mu$ and $\sigma$ have units of km s$^{-1}$.}\n' \
        + '\n' \
        + '\\tablenotetext{a}{The fitted amplitudes of slit M0 are significantly different from the other observations because this observation was taken through cloud cover that prevented an accurate fluxing. See Table \\ref{tab:mage_log} for more information about the observing conditions.}\n' \
        + '\n' \
        + '\end{deluxetable}'

    # Save the table
    f = open(f'{results}/tables/lya_best_fit_model_parameters_table.tex', 'w', encoding='utf-8')
    f.write(table)
    f.close()

In [ ]:
fit()

In [9]:
correlate()

In [31]:
tabulate()

In [ ]:
plot()